## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-08-15-14-52-48 +0000,nyt,Flash Floods in Pakistan Kill at Least 150 Peo...,https://www.nytimes.com/2025/08/15/world/asia/...
1,2025-08-15-14-52-00 +0000,wsj,Washington Sues Trump Administration to Stop P...,https://www.wsj.com/us-news/attorney-general-n...
2,2025-08-15-14-50-39 +0000,nyt,Live Updates: Trump Flies to Alaska for High-S...,https://www.nytimes.com/live/2025/08/15/world/...
3,2025-08-15-14-50-07 +0000,nyt,DC Lawsuit Challenges Trump’s Federal Takeover...,https://www.nytimes.com/2025/08/15/us/washingt...
4,2025-08-15-14-45-11 +0000,nyt,Fed Chair Faces Scrutiny — and Bats — at Major...,https://www.nytimes.com/2025/08/15/business/fe...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2333/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
9,trump,64
20,putin,30
72,will,17
19,summit,15
16,alaska,14


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
17,2025-08-15-13-50-27 +0000,nyt,"How Trump’s Meeting With Putin Could Unfold, a...",https://www.nytimes.com/2025/08/15/us/politics...,186
28,2025-08-15-13-25-54 +0000,nypost,Putin’s foreign minister trolls Ukraine by wea...,https://nypost.com/2025/08/15/world-news/putin...,172
167,2025-08-15-01-00-00 +0000,wsj,"President Trump, by meeting Vladimir Putin on ...",https://www.wsj.com/politics/national-security...,159
127,2025-08-15-06-37-53 +0000,cbc,Trump tries to turn the page on Ukraine war wi...,https://www.cbc.ca/news/politics/trump-putin-r...,153
123,2025-08-15-07-00-55 +0000,wapo,"How Putin, ex-KGB officer, will seek to sway T...",https://www.washingtonpost.com/world/2025/08/1...,150


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
17,186,2025-08-15-13-50-27 +0000,nyt,"How Trump’s Meeting With Putin Could Unfold, a...",https://www.nytimes.com/2025/08/15/us/politics...
139,76,2025-08-15-04-27-14 +0000,nypost,AG Pam Bondi declares DC ‘will not remain a sa...,https://nypost.com/2025/08/15/us-news/ag-pam-b...
201,57,2025-08-14-22-26-00 +0000,wsj,California Democrats are planning their own re...,https://www.wsj.com/politics/policy/gavin-news...
309,50,2025-08-14-15-59-01 +0000,nypost,Trump reveals the odds of Putin summit ending ...,https://nypost.com/2025/08/14/us-news/trump-gi...
8,41,2025-08-15-14-35-01 +0000,startribune,Washington sues to block Trump's federal takeo...,https://www.startribune.com/new-lawsuit-challe...
93,39,2025-08-15-09-30-00 +0000,wsj,Tariffs haven’t caused inflation to surge as m...,https://www.wsj.com/economy/trade/trump-tariff...
164,34,2025-08-15-01-11-32 +0000,latimes,An L.A. high school senior was walking his dog...,https://www.latimes.com/california/story/2025-...
167,33,2025-08-15-01-00-00 +0000,wsj,"President Trump, by meeting Vladimir Putin on ...",https://www.wsj.com/politics/national-security...
168,32,2025-08-15-01-00-00 +0000,wsj,The nine law firms that promised President Tru...,https://www.wsj.com/politics/policy/trump-pro-...
296,32,2025-08-14-16-23-09 +0000,nypost,"Toronto film festival reverses course, will ‘e...",https://nypost.com/2025/08/14/world-news/tiff-...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
